In [0]:
%sql
drop schema if exists dkushari_uc.mars_schema_1 cascade;
create schema if not exists dkushari_uc.mars_schema_1 
managed location 'abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars1';

### Delta Sharing Scenario

##### Set up tables

In [0]:
%sql
drop table if exists dkushari_uc.mars_schema_1.people_delta;
create table if not exists dkushari_uc.mars_schema_1.people_delta
partitioned by (gender)
as
select * 
from parquet.`dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.parquet/`
where gender = 'M' 
limit 10;

num_affected_rows,num_inserted_rows


In [0]:
%sql
insert into dkushari_uc.mars_schema_1.people_delta
select * 
from parquet.`dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.parquet/`
where gender = 'F' 
limit 10;

num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
desc extended dkushari_uc.mars_schema_1.people_delta;

col_name,data_type,comment
id,int,null
firstName,string,null
middleName,string,null
lastName,string,null
gender,string,null
birthDate,timestamp,null
ssn,string,null
salary,int,null
# Partition Information,,
# col_name,data_type,comment


In [0]:
%sql
select gender, count(*) from dkushari_uc.mars_schema_1.people_delta group by gender;

gender,count(1)
F,10
M,10


In [0]:
%sql
select * from dkushari_uc.mars_schema_1.people_delta where gender = 'M' limit 3;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
5000001,Jasper,Gil,Neubigin,M,1961-02-20T05:00:00Z,900-26-1632,71908
5000002,Robert,Lindsay,Kipling,M,1986-01-02T05:00:00Z,971-15-9711,73156
5000003,Abdul,Mickey,Pachta,M,1956-01-28T05:00:00Z,935-93-3364,71366


In [0]:
%sql
select * from dkushari_uc.mars_schema_1.people_delta where gender = 'F' limit 3;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
1,Pennie,Carry,Hirschmann,F,1955-07-02T04:00:00Z,981-43-9345,56172
2,An,Amira,Cowper,F,1992-02-08T05:00:00Z,978-97-8086,40203
3,Quyen,Marlen,Dome,F,1970-10-11T04:00:00Z,957-57-8246,53417


##### Set up Delta Sharing infra

In [0]:
%sql
CREATE RECIPIENT if not exists own_azure_recipient
USING ID 'azure:westus:d5a6d8a8-4922-4b07-9b10-c6bdce475b80'
COMMENT "For testing Mars use cases";

info_name,info_value


In [0]:
%sql
DESCRIBE RECIPIENT own_azure_recipient;

info_name,info_value
recipient_name,own_azure_recipient
authentication_type,DATABRICKS
owner,dipankar.kushari@databricks.com
created_by,dipankar.kushari@databricks.com
created_at,2025-04-08 15:34:44.402
updated_by,dipankar.kushari@databricks.com
updated_at,2025-06-30 16:02:49.026
properties,"{""databricks.name"":""own_azure_recipient"",""databricks.accountId"":""827e3e09-89ba-4dd2-9161-a3301d0f21c0"",""databricks.metastoreId"":""d5a6d8a8-4922-4b07-9b10-c6bdce475b80""}"
comment,For testing Mars use cases.
cloud,azure


In [0]:
%sql
CREATE SHARE IF NOT EXISTS mars_share
COMMENT "For testing Mars use cases";

info_name,info_value
share_name,mars_share
owner,dipankar.kushari@databricks.com
created_at,2025-06-30 17:17:35.246
created_by,dipankar.kushari@databricks.com
updated_at,2025-06-30 17:17:35.246
updated_by,dipankar.kushari@databricks.com
comment,For testing Mars use cases


In [0]:
%sql
ALTER SHARE mars_share ADD TABLE dkushari_uc.mars_schema_1.people_delta
COMMENT "Sharing a full partitioned table with full history."
as mars_schema_1.full_people_delta
WITH HISTORY;

In [0]:
%sql
SHOW GRANTS TO RECIPIENT own_azure_recipient;

share,privilege


In [0]:
%sql
GRANT SELECT ON SHARE mars_share TO RECIPIENT own_azure_recipient;

In [0]:
%sql
SHOW GRANTS TO RECIPIENT own_azure_recipient;

share,privilege
mars_share,SELECT


In [0]:
%sql
CREATE OR REPLACE VIEW dkushari_uc.mars_schema_1.people_redacted AS
SELECT
  id,
  firstName,
  middleName,
  lastName,
  gender,
  birthDate,
  ssn,
  salary
FROM dkushari_uc.mars_schema_1.people_delta
WHERE
CASE
    WHEN is_account_group_member('mars_test_group') THEN true
    WHEN is_account_group_member('mars_male_group') THEN gender = 'M'
    WHEN is_account_group_member('mars_female_group') THEN gender = 'F'
    ELSE false
END;

In [0]:
%sql
ALTER SHARE mars_share ADD view dkushari_uc.mars_schema_1.people_redacted
COMMENT "Sharing dynamic view."
as mars_schema_1.people_redacted;


warning_text
Warning: Shared views for Databricks-to-Databricks Delta Sharing only supports a subset of built-in functions. See: https://docs.databricks.com/en/delta-sharing/ shared-view-functions-data-databricks.html


In [0]:
%sql
SHOW GROUPS WITH USER `dipankar.kushari@databricks.com`;

name,directGroup
team.fieldeng-all,true
account users,true
admins,true


### Right now the user does not belong to any of the groups mentioned in DV definition. So, the "Else" logic is enforced.

In [0]:
%sql
select * from dkushari_uc.mars_schema_1.people_redacted;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary


### Lets add the user to the mars_test_group - both male and female records will be accessible

In [0]:
%sql
SHOW GROUPS WITH USER `dipankar.kushari@databricks.com`;

name,directGroup
account users,true
team.fieldeng-all,true
admins,true
mars_test_group,true


In [0]:
%sql
select * from dkushari_uc.mars_schema_1.people_redacted;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
1,Pennie,Carry,Hirschmann,F,1955-07-02T04:00:00.000Z,981-43-9345,56172
2,An,Amira,Cowper,F,1992-02-08T05:00:00.000Z,978-97-8086,40203
3,Quyen,Marlen,Dome,F,1970-10-11T04:00:00.000Z,957-57-8246,53417
4,Coralie,Antonina,Marshal,F,1990-04-11T04:00:00.000Z,963-39-4885,94727
5,Terrie,Wava,Bonar,F,1980-01-16T05:00:00.000Z,964-49-8051,79908
6,Chassidy,Concepcion,Bourthouloume,F,1990-11-24T05:00:00.000Z,954-59-9172,64652
7,Geri,Tambra,Mosby,F,1970-12-19T05:00:00.000Z,968-16-4020,38195
8,Patria,Nancy,Arstall,F,1985-01-02T05:00:00.000Z,984-76-3770,102053
9,Terese,Alfredia,Tocque,F,1967-11-17T05:00:00.000Z,967-48-7309,91294
10,Wava,Lyndsey,Jeandon,F,1963-12-30T05:00:00.000Z,997-82-2946,56521
